# 01 PostgreSQL tutorial

To install PostgreSQL you can either use a package manager, e.g.

`brew install postgresql`

or (use in Ubuntu)

`apt-get install postgresql-11`

More options are available under https://www.postgresql.org/download/ (binary packages).

Another option is to download Postgres and (manually) install/use it. If you want to work on the department machines, this is what you'll need to do. I.e. 

``


The first step is to create a new database (here called cs6) using the createdb command in the bash shell:

`createdb cs6`


Note: you can use man with all PostgreSQL commands! E.g. `man createdb` or `man psql`

## 01.01 PostgreSQL shell
To connect to the `cs6` database and to start the (interactive) PostgreSQL shell, run `psql cs6`.

Quit/exit the shell by typing `\q`.

Typing `help` shows a list of commands.

In particular you can use 
- `\?` to show all available psql-shell commands.
- `\h <SQL>` to get help on the syntax of a SQL statement, e.g. `\h SELECT`


To edit the last command, use `\e`. This starts up `vim` or whatever editor you configured to work with your PostgreSQL instance.


You can also directly run a SQL query or command using `psql -c '<cmd>' cs6`

Resetting the notebook:

In [ ]:
!dropdb cs6 && createdb cs6

## 01.02 Creating a table
Let's create a table for pokemon! 

In [ ]:
!psql -c 'CREATE TABLE pokemon(name VARCHAR(128) NOT NULL, \
height_ft DECIMAL NOT NULL, weight_lbs DECIMAL NOT NULL, \
category VARCHAR(128) NOT NULL, PRIMARY KEY (name));' cs6

We can get an overview of all defined tables via `\dt`

In [ ]:
!psql -c '\dt' cs6

To see what the table looks like, let's use `\d`

In [ ]:
!psql -c '\d pokemon' cs6

## 01.03 Inserting data
After creating an empty table, we want to populate it with values. Check <https://www.pokemon.com/us/pokedex> for data.

Note that strings need to be quoted using `'....'`


<table>
    <tr>
<td><img src="https://assets.pokemon.com/assets/cms2/img/pokedex/full/001.png" width="100px"></td>
<td><img src="https://assets.pokemon.com/assets/cms2/img/pokedex/full/007.png" width="100px"></td>
<td><img src="https://assets.pokemon.com/assets/cms2/img/pokedex/full/004.png" width="100px"></td>
    </tr>
</table>

In [ ]:
!psql -c "INSERT INTO pokemon (name, category, height_ft, weight_lbs) VALUES ('Bulbasaur', 'Seed', 2.04, 15.2);" cs6

In [ ]:
!psql -c "INSERT INTO pokemon (name, category, height_ft, weight_lbs) VALUES ('Squirtle', 'Tiny Turtle', 1.08, 19.8);" cs6

In [ ]:
!psql -c "INSERT INTO pokemon (name, category, height_ft, weight_lbs) VALUES ('Charmander', 'Lizard', 2.00, 18.7);" cs6

## 01.04 Retrieving data
To retrieve data the `SELECT` command is used. It can be used to retrieve only a subset of rows and columns (filtering). Use `*` to retrieve everything.

In [ ]:
!psql -c "SELECT * FROM pokemon;" cs6

In [ ]:
!psql -c "SELECT name FROM pokemon WHERE height_ft > 2;" cs6

In [ ]:
!psql -c "SELECT category, name FROM pokemon WHERE height_ft > 2;" cs6

There are many ways how the result of a `SELECT` statement can be manipulated. That's what makes `SQL` so powerful!

In [ ]:
!psql -c "SELECT * FROM pokemon ORDER BY name;" cs6

## 01.05 Updating data
With the `update` statement, rows can be updated.

Assume we made a mistake when entering the height of Caterpie.

<img src="https://assets.pokemon.com/assets/cms2/img/pokedex/full/010.png" width="100px">

In [ ]:
!psql -c "INSERT INTO pokemon (name, category, height_ft, weight_lbs) \
VALUES ('Caterpie', 'Worm', 42, 6.4);" cs6

In [ ]:
!psql -c "SELECT * FROM pokemon WHERE name LIKE 'Cater%';" cs6

In [ ]:
!psql -c "UPDATE pokemon SET height_ft=1.0 WHERE name='Caterpie'" cs6

In [ ]:
!psql -c "SELECT * FROM pokemon WHERE name LIKE 'Cater%';" cs6

## 01.06 Removing rows
With the `delete` command individual rows can be removed.

In [ ]:
!psql -c "DELETE FROM pokemon WHERE name='Caterpie'" cs6

In [ ]:
!psql -c "SELECT * FROM pokemon" cs6

# 02 Connecting python to PostgreSQL

To use postgresql in a python module, we'll use psycopg2 (http://initd.org/psycopg/)

See http://initd.org/psycopg/docs/usage.html for a tutorial on how to use this package.

In [ ]:
!psql -c "DELETE FROM pokemon WHERE name='Pikachu';" cs6

In [ ]:
import psycopg2

In [ ]:
# connect to existing Postgresql db

conn = psycopg2.connect("dbname=cs6")

In [ ]:
# open cursor to perform db operations
# ==> i.e. will start a new transaction
cur = conn.cursor()

In [ ]:
# execute SQL statement
# use fetchall to retrieve all results
cur.execute("SELECT * FROM pokemon")
cur.fetchall()

In [ ]:
# execute SQL statement
# use fetchone to retrieve one row
cur.execute("SELECT * FROM pokemon")
cur.fetchone()

Of course working with a database gets only interesting if we can work with python objects. Psycopg2 offers soem native mappers of python types to SQL types. A list can be found here <http://initd.org/psycopg/docs/usage.html#adaptation-of-python-values-to-sql-types>

In [ ]:
# inserting a new pokemon via python

name = 'Pikachu'
height = 1.04
weight = 13.2
category = 'Mouse'


# PASS PARAMETERS LIKE THIS
# always use %s
query = cur.mogrify("INSERT INTO pokemon (name, category, height_ft, weight_lbs) \
VALUES (%s, %s, %s, %s);", (name, category, height, weight))
query

In [ ]:
# perform transaction & commit
cur.execute(query)
conn.commit()

In [ ]:
!psql -c "SELECT * FROM pokemon" cs6

In [ ]:
# close connection
cur.close()
conn.close()

## 2.02 Escaping user input - IT'S VERY IMPORTANT!

NOTE: Never, never construct queries from user input without escaping properly. 
=> To not make mistakes, use the safe functions provided by psycopg2!

Why?

Let's see some example:

In [ ]:
!psql -c "DELETE FROM pokemon WHERE name='Pikachu';" cs6

In [ ]:
conn = psycopg2.connect("dbname=cs6")
cur = conn.cursor()

In [ ]:
# Assume these are coming from a query form: We can destroy the database!

name = "'Pikachu'"
height = 1.04
weight = '13.2); SELECT u.* FROM pg_catalog.pg_user u; (SELECT * FROM pg_catalog.pg_user ' # attacker entered this into a form!!!
category = "'Mouse'"

In [ ]:
query = "INSERT INTO pokemon (name, category, height_ft, weight_lbs) \
VALUES ({}, {}, {}, {});".format(name, category, height, weight)

query

In [ ]:
cur.execute(query)
conn.commit()

In [ ]:
# this is output somewhere on a website...
cur.fetchall()

In [ ]:
!psql -c '\d pg_catalog.pg_user' cs6

In [ ]:
cur.close()
conn.close()

This is called SQL injection. You can read more on this topic under <https://portswigger.net/web-security/sql-injection>